#### Задача 1 (5 баллов).

Реализуйте с помощью классов базу данных для ветеринарной клиники: нам нужно хранить сведения о питомцах, их хозяевах и датах приема. Видимо, нам понадобятся два класса: для владельцев животных и для самих животных. У владельца мы храним имя, фамилию и номер телефона. У животного пусть будет кличка, возраст и дополнительный атрибут "записан на прием", в котором будет храниться информация о дате следующего приема (если ее нет - None). База данных должна храниться в бинарном файле (используйте pickle или dill). При запуске программы должны быть доступны несколько действий: завести нового клиента, завести новое животное для существующего клиента и записать животное на прием (или отметить, что прием прошел и новой записи нет). Сами экземпляры клиентов хранятся просто в списке.

Примечание: в атрибутах могут храниться не только строки, но и списки, но не забывайте об их особенностях.

In [ ]:
import pickle
from dataclasses import dataclass

class Database:
    def __init__(self):
        self.owners: list[Owner] = []

class Owner:
    def __init__(self, first_name: str, last_name: str,
                 phone_number: str):
        self.first_name: str = first_name
        self.last_name: str = last_name
        self.phone_number: str = phone_number
        self.pets: list[Pet] = []

    def __str__(self):
        return f"Владелец: {self.first_name} {self.last_name}\n\
Телефон: {self.phone_number}\n\
Питомцы: {self.pets}"


class Pet:
    def __init__(self, name: str, age: int,
                 at_appointment: str = None):
        self.name: str = name
        self.age: int = age
        self.at_appoitment: str | None = at_appointment

    def __str__(self):
        return f"Питомец: {self.name},\n\t\
Возраст: {self.age}\n\t\
Прием: {self.at_appoitment}\n\t"

    def __repr__(self):
        return f"Питомец: {self.name},\n\t\
Возраст: {self.age}\n\t\
Прием: {self.at_appoitment}\n\t"


def choose_owner(db) -> int:
    print("Выберите клиента цифрой:")
    for i, owner in enumerate(db.owners):
        print(f"{i + 1}: {owner.first_name} {owner.last_name}")
    while True:
        try:
            choise = int(input())
            if choise > len(db.owners):
                print("Нет такого клиента, попробуйте еще раз")
                continue
            return choise - 1
        except ValueError:
            print("Вы ввели не цифру, попробуйте еще раз")
            continue

def choose_pet(owner: Owner) -> int:
    print("Выберите питомца цифрой:")
    for i, pet in enumerate(owner.pets):
        print(f"{i + 1}: {pet.name}")
    while True:
        try:
            choise = int(input())
            if choise > len(owner.pets):
                print("Нет такого клиента, попробуйте еще раз")
                continue
            break
        except ValueError:
            print("Вы ввели не цифру, попробуйте еще раз")
    return choise - 1

In [1]:
import os
import pickle


running_state = True
path = os.path.join("HW04", "database.pkl")

if os.path.exists(path):
    db = pickle.load(open(path, "rb"))
else:
    db = Database()

while running_state:
    user_input = input("Введите команду:\n")
    match user_input.lower():
        case "помощь":
            print("Команды:\n\t\
Помощь\n\t\
Ввести клиента\n\t\
Добавить питомца\n\t\
Удалить питомца\n\t\
Вывести\n\t\
Выйти")
        case "выйти":
            pickle.dump(db, open(path, "wb"))
            running_state = False
        case "ввести клиента":
            user_input = input("Введите имя клиента, фамилию,\
и номер телефона через запятую:\n")
            owner_data = user_input.split(", ")
            owner = Owner(*owner_data)
            db.owners.append(owner)
        case "добавить питомца":
            choise = choose_owner(db)
            print("Введите имя питомца, возраст и дату осмотра (если есть) через запятую")
            user_input = input()
            pet_data = user_input.split(", ")
            pet = Pet(*pet_data)
            db.owners[choise].pets.append(pet)
        case "удалить питомца":
            choise = choose_owner(db)
            pet_choise = choose_pet(db.owners[choise])
            db.owners[choise].pets.pop(pet_choise)
        case "изменить дату осмотра":
            choise = choose_owner(db)
            pet_choise = choose_pet(db.owners[choise])
            user_input = input("Введите дату осмотра или нажмите энтер для удаления")
            if user_input == "\n":
                user_input = None
            db.owners[choise].pets[pet_choise].at_appoitment = user_input
        case "вывести":
            for owner in db.owners:
                print()
                print(owner)

Введите команду:

Владелец: Иван Иванович
Телефон: 89998887766
Питомцы: [Питомец: Муся,
	Возраст: 10
	Прием: 25.12.2024
	, Питомец: Тузик,
	Возраст: 4
	Прием: None
	]

Владелец: Артем Артемович
Телефон: 89991112233
Питомцы: [Питомец: Гарик,
	Возраст: 3
	Прием: None
	]
Введите команду:
Выберите клиента цифрой:
1: Иван Иванович
2: Артем Артемович
Выберите питомца цифрой:
1: Муся
2: Тузик
Введите команду:

Владелец: Иван Иванович
Телефон: 89998887766
Питомцы: [Питомец: Муся,
	Возраст: 10
	Прием: 25.12.2024
	]

Владелец: Артем Артемович
Телефон: 89991112233
Питомцы: [Питомец: Гарик,
	Возраст: 3
	Прием: None
	]
Введите команду:


#### Задача 2 (5 баллов).

Хотим написать программу - базу данных для библиотеки. Нам понадобится класс "книга", который будет содержать автора, название, жанр и количество страниц; а также класс "библиотека", в котором в атрибутах будут сидеть все наши книги. В библиотеку будут поступать запросы: нужно будет выдавать перечень всех книг одного автора, перечень всех книг конкретного жанра, а также проверять, что книга такого автора и с таким названием есть (вам хорошо помогут генераторные выражения для всех этих вещей, а еще можно переопределить магический метод \_\_eq\_\_ у класса "книга" - он должен возвращать bool, а внутри него можно сравнивать атрибуты экземпляров self и other - и проверять наличие объекта класса "книга" в списке). Магический метод \_\_eq\_\_ неявным образом вызывается, когда вы сравниваете два объекта: например, сравниваете две книжки. Он будет неявно вызываться и тогда, когда вы проверяете наличие какого-то объекта в списке таких же объектов с помощью оператора in. 

In [ ]:
# your code here

#### Задача 3 (5 баллов). 

Представим себе, что мы хотим написать игрушку, в которой главный персонаж ходит по карте и с каждым броском кубика ему выпадают разнообразные предметы, с которыми он может взаимодействовать. Нам, очевидно, понадобится хранить все результаты взаимодействия в классе персонажа, а также написать хотя бы три различных предмета, которые умеют делать разные вещи (пусть будет волшебное зелье, книжка, прибавляющая мудрость, и медвежий капкан; можете что-то свое придумать). У персонажа есть свои характеристики, которые тоже могут влиять на результат взаимодействия с вещами (очень тупой персонаж может не смочь прочитать книжку...). 

Бонусный балл: напишите класс для карты, у которого будет метод с собственно циклом, где кидается наш "кубик" и персонаж перемещается, подбирая предметы. 

Для рандомизации используем библиотеку random. Также можно использовать функцию sleep из библиотеки time: эта функция заставляет компьютер ждать x секунд. 

In [ ]:
from random import randint # функция генерирует любое случайное число от х до у: randint(x, y)
from time import sleep # sleep(1) - ждать 1 секунду

class Hero:
    def __init__(self, *args): # замените *args на свои параметры
        # your code here
        
    def __repr__(self):
        # your code here
    
    def interact(self, obj):
        # your code here
        
class Potion:
    def __init__(self, *args):
        # your code here 
        
    def __repr__(self):
        #your code here 
        
...

class Map:
    def __init__(self):
        self.hero = Hero()
        # your code here 
        
    def travel(self):
        # your code here